In [1]:
#50

import pandas as pd

#参考にしたサイト
#(50.1)https://note.nkmk.me/python-pandas-read-csv-tsv/

df = pd.read_table("chap06/newsCorpora.csv", header=None, names=("ID","TITLE","URL","PUBLISHER","CATEGORY","STORY","HOSTNAME","TIMESTAMP"))

In [2]:
#(50.2)https://note.nkmk.me/python-pandas-query/

df_50_2 = df[df["PUBLISHER"].isin(["Reuters","Huffington Post","Businessweek","Contactmusic.com","Daily Mail"])]

In [3]:
#(50.3)https://note.nkmk.me/python-pandas-random-sort-shuffle/

df_50_3 = df_50_2.sample(frac=1,random_state=0)

In [4]:
#(50.4)

from sklearn.model_selection import train_test_split

##まず80%(学習データ)と20%(検証データ+評価データ)にわける
train,valid_test = train_test_split(df_50_3,test_size=0.2,random_state=0)

##つぎに、20%を検証データと評価データにわける
valid,test = train_test_split(valid_test,test_size=0.5,random_state=0)

##念のため大きさを確認する
print("train: %d" % (len(train)))
print("valid: %d" % (len(valid)))
print("test: %d" % (len(test)))

##ファイル書き出し
##https://note.nkmk.me/python-pandas-to-csv/
train.to_csv("chap06/train.txt",columns=["CATEGORY","TITLE"],sep="\t",header=False,index=False)
valid.to_csv("chap06/valid.txt",columns=["CATEGORY","TITLE"],sep="\t",header=False,index=False)
test.to_csv("chap06/test.txt",columns=["CATEGORY","TITLE"],sep="\t",header=False,index=False)

train: 10672
valid: 1334
test: 1334


In [5]:
##各カテゴリの事例数確認
##https://note.nkmk.me/python-pandas-value-counts/
print("*train")
print(train["CATEGORY"].value_counts())
print("*valid")
print(valid["CATEGORY"].value_counts())

*train
b    4473
e    4233
t    1235
m     731
Name: CATEGORY, dtype: int64
*valid
b    583
e    533
t    123
m     95
Name: CATEGORY, dtype: int64


In [6]:
#51
#記事のタイトルに出てくる単語をカウントして特徴量にする
#参考にしたサイト
#https://qiita.com/fujin/items/b1a7152c2ec2b4963160

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
vec_count = CountVectorizer()

##学習データ
vec_count.fit(train["TITLE"])
train_feature = vec_count.transform(train["TITLE"])
#print(vec_count.vocabulary_)
#print(len(vec_count.vocabulary_))
np.savetxt("chap06/train_feature.txt", train_feature.toarray(), fmt="%d")

##検証データ
vec_count.fit(valid["TITLE"])
valid_feature = vec_count.transform(valid["TITLE"])
#print(vec_count.vocabulary_)
#print(len(vec_count.vocabulary_))
np.savetxt("chap06/valid_feature.txt", valid_feature.toarray(), fmt="%d")

##評価データ
vec_count.fit(test["TITLE"])
test_feature = vec_count.transform(test["TITLE"])
#print(vec_count.vocabulary_)
#print(len(vec_count.vocabulary_))
np.savetxt("chap06/test_feature.txt", test_feature.toarray(), fmt="%d")

In [7]:
# 52
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(train_feature, train['CATEGORY'])

/Users/ayano/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)